## Loading The Dataset 

In [56]:
import pandas as pd
df=pd.read_csv("E:\\R.P 2\\Data-Credit Card Transactions Fraud Detection 2019–2020\\fraudTrain.csv")
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,1/1/2019 0:00,2.700000e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",3/9/1988,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,1/1/2019 0:00,6.300000e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,6/21/1978,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,1/1/2019 0:00,3.890000e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1/19/1962,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,1/1/2019 0:01,3.530000e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1/12/1967,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,1/1/2019 0:03,3.760000e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,3/28/1986,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [57]:
df.shape

(1048575, 23)

## Preprocessing For Column "trans_date_trans_time"

## "trans_date_trans_time" Converting Format & adding new Columns And Time Gap 

In [58]:
# 1. Convert to DateTime
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])


In [59]:
# 2. Extract Hour, Day, Month, Weekday
df['hour'] = df['trans_date_trans_time'].dt.hour
df['day'] = df['trans_date_trans_time'].dt.day
df['month'] = df['trans_date_trans_time'].dt.month
df['weekday'] = df['trans_date_trans_time'].dt.weekday  # Monday=0



In [60]:
# 3. Time gap per card (in minutes)
df = df.sort_values(by=['cc_num','trans_date_trans_time'])
df['time_gap'] = df.groupby('cc_num')['trans_date_trans_time'].diff().dt.total_seconds() / 60



In [61]:
# Missing gaps (first transaction per card) → fill with 0
df['time_gap'] = df['time_gap'].fillna(0)
df.sample(5)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,hour,day,month,weekday,time_gap
684134,684134,2019-10-19 08:22:00,3.460000e+14,fraud_Cassin-Harvey,grocery_net,96.70,Donna,Moreno,F,32301 Albert River Suite 364,...,6cf646d0b15a37ca8dc0d3b25e006885,1350634956,38.551122,-81.318741,0,8,19,10,5,321.0
830463,830463,2019-12-10 21:35:00,3.600000e+15,fraud_Kutch-Wilderman,home,32.51,Ashley,Lopez,F,9333 Valentine Point,...,3f862dd3d8a3db5daaea185f8a55fdb5,1355175312,41.383151,-72.719687,0,21,10,12,1,34.0
782730,782730,2019-11-30 20:12:00,3.540000e+15,"fraud_Graham, Hegmann and Hammes",food_dining,52.73,Christian,Johns,M,892 Solis Neck,...,7ce614377cbc8ea4e93e34867e05b18f,1354306336,44.270427,-92.730614,0,20,30,11,5,13.0
372091,372091,2019-06-21 14:54:00,4.340000e+15,fraud_Harris Group,food_dining,57.45,Kevin,Elliott,M,1852 Vincent Isle Suite 328,...,6d4c903efd58d6c646f6ef7eafbae768,1340290471,42.832281,-76.066007,0,14,21,6,4,311.0
428396,428396,2019-07-11 10:57:00,3.490000e+14,"fraud_Mosciski, Gislason and Mertz",grocery_pos,131.72,Megan,Murphy,F,53696 Laura Radial,...,b7185a45845c1e6c33232d312e4a8a52,1342004274,39.490540,-98.214300,0,10,11,7,3,168.0


In [62]:
df.groupby('is_fraud')['time_gap'].describe()


,count,mean,std,min,25%,50%,75%,max
is_fraud,,,,,,,,
0,1042569.0,187.480501,421.411011,0.0,15.0,52.0,176.0,20095.0
1,6006.0,126.903430,305.115645,0.0,10.0,29.0,86.0,8388.0


In [63]:
df[df['time_gap'] < 5]   # transactions within 5 minutes


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,hour,day,month,weekday,time_gap
1017,1017,2019-01-01 12:47:00,6.041621e+10,"fraud_Jones, Sawayn and Romaguera",misc_net,7.27,Mary,Diaz,F,9886 Anita Drive,...,98e3dcf98101146a577f85a34e58feec,1325422035,43.974711,-109.741904,0,12,1,1,1,0.0
2726,2726,2019-01-02 08:47:00,6.041621e+10,fraud_Luettgen PLC,gas_transport,82.08,Mary,Diaz,F,9886 Anita Drive,...,95f514bb993151347c7acdf8505c3d62,1325494056,42.961335,-109.157564,0,8,2,1,2,3.0
14342,14342,2019-01-08 23:24:00,6.041621e+10,"fraud_Effertz, Welch and Schowalter",entertainment,81.48,Mary,Diaz,F,9886 Anita Drive,...,05e7c66f7be34122cb99a04f3ef6b54c,1326065083,43.594569,-109.361063,0,23,8,1,1,4.0
28631,28631,2019-01-17 19:24:00,6.041621e+10,"fraud_Nitzsche, Kessler and Wolff",shopping_pos,1.84,Mary,Diaz,F,9886 Anita Drive,...,8755ece3a44e544c5022c676f317ef6d,1326828257,42.222055,-109.309681,0,19,17,1,3,1.0
106628,106628,2019-03-02 22:10:00,6.041621e+10,"fraud_Windler, Goodwin and Kovacek",home,261.79,Mary,Diaz,F,9886 Anita Drive,...,a2549382981f0b230d3e7dde8d58a699,1330726259,42.204230,-109.341230,1,22,2,3,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920748,920748,2019-12-30 19:58:00,4.990000e+18,fraud_Bartoletti and Sons,personal_care,13.22,Benjamin,Kim,M,920 Patrick Light,...,f91a8c82632f2b76bcc3d45e1e7754d2,1356897529,41.270587,-89.572224,0,19,30,12,0,1.0
930193,930193,2020-01-04 16:35:00,4.990000e+18,"fraud_Parker, Nolan and Trantow",entertainment,21.77,Benjamin,Kim,M,920 Patrick Light,...,0b13f304f67fc48a3dd0837a1075b2a1,1357317304,40.443260,-89.003534,0,16,4,1,5,1.0
1019094,1019094,2020-02-24 19:32:00,4.990000e+18,"fraud_McDermott, Osinski and Morar",home,60.59,Benjamin,Kim,M,920 Patrick Light,...,4b3816926b15ab2fc5c87e26dfa64a9e,1361734362,41.634416,-88.360986,0,19,24,2,0,4.0
1044168,1044168,2020-03-09 07:37:00,4.990000e+18,fraud_Brown PLC,misc_net,6.89,Benjamin,Kim,M,920 Patrick Light,...,295581b102019d12e867ae2067c1805f,1362814623,40.874160,-90.129467,0,7,9,3,0,1.0


## Preprocessing For Column "Unnamed: 0"

In [64]:
df = df.drop('Unnamed: 0', axis=1)


## Preprocessing For Column "cc_num"

In [65]:
df['cc_num'] = df['cc_num'].astype(str)


## Preprocessing For Column "merchant"

In [66]:
df['merchant'] = df['merchant'].astype(str)


## Preprocessing For Column "category"

In [67]:
df['category'] = df['category'].astype(str) 


## Preprocessing For Column "amt"

In [68]:
df['amt'] = df['amt'].astype(float)


## Preprocessing For Column "first""last" (Droped its just Customer name not usefull Feature )


In [69]:
df = df.drop(['first','last'], axis=1)

## Preprocessing For Column "gender" 

In [70]:
df['gender'] = df['gender'].astype(str)


## Preprocessing For Column "street" (Droped not usefull it is only Cutomer adress)

In [71]:
df = df.drop('street', axis=1)


## Preprocessing For Column "city, state"

In [72]:
df['city'] = df['city'].astype(str)
df['state'] = df['state'].astype(str)


## Preprocessing For Column "zip"

In [73]:
df['zip'] = df['zip'].astype(str)


## Preprocessing For Column "lat, long" 

In [74]:
df['lat'] = df['lat'].astype(float)
df['long'] = df['long'].astype(float)


## Preprocessing For Column "city_pop"


In [75]:
df['city_pop'] = df['city_pop'].astype(int)


## Preprocessing For Column "job"

In [76]:
df['job'] = df['job'].astype(str)


## Preprocessing For Column "dob"  using this Created Age 

In [77]:
df['dob'] = pd.to_datetime(df['dob'])
df['age'] = df['trans_date_trans_time'].dt.year - df['dob'].dt.year

## Preprocessing For Column "trans_num" (Droped not usefull overall)

In [78]:
df = df.drop('trans_num', axis=1)

## Preprocessing For Column "unix_time"

In [79]:
df['unix_time'] = df['unix_time'].astype(int)


## Preprocessing For Column "merch_lat, merch_long"

In [80]:
df['merch_lat'] = df['merch_lat'].astype(float)
df['merch_long'] = df['merch_long'].astype(float)


## Preprocessing For Column "is_fraud"

In [81]:
df['is_fraud'] = df['is_fraud'].astype(int)


In [85]:
df.isnull().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1048575 entries, 1017 to 1047587
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   trans_date_trans_time  1048575 non-null  datetime64[ns]
 1   cc_num                 1048575 non-null  object        
 2   merchant               1048575 non-null  object        
 3   category               1048575 non-null  object        
 4   amt                    1048575 non-null  float64       
 5   gender                 1048575 non-null  object        
 6   city                   1048575 non-null  object        
 7   state                  1048575 non-null  object        
 8   zip                    1048575 non-null  object        
 9   lat                    1048575 non-null  float64       
 10  long                   1048575 non-null  float64       
 11  city_pop               1048575 non-null  int64         
 12  job                    1048575

## Checking Duplicate Transitions 

In [83]:
df.duplicated().sum()


np.int64(0)

## Class Imbalance Check 

In [84]:
df['is_fraud'].value_counts(normalize=True)*100


is_fraud
0    99.427223
1     0.572777
Name: proportion, dtype: float64

## Exporting Prprocessed Data to CSV File 

In [86]:
# Assuming 'df' is your final preprocessed dataframe
df.to_csv('cleaned_data.csv', index=False)
print("File exported successfully as cleaned_data.csv") 



File exported successfully as cleaned_data.csv
